<a href="https://colab.research.google.com/github/jainMaurya/UCS547-Accelerated-Data-Science/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1: Identify !, %, and %% used in Google Colab

In [24]:
%time x=sum(range(1000000))

CPU times: user 121 ms, sys: 26.3 ms, total: 147 ms
Wall time: 148 ms


In [25]:
%%writefile hello.cu
#include<stdio.h>
int main(){
printf("Hello World\n");
return 0;
}

Writing hello.cu


In [26]:
!nvcc -arch=sm_75 hello.cu -o hello
!./hello

Hello World


Q2: Key nvidia-smi commands with multiple options

In [27]:
!nvidia-smi

Fri Feb 20 00:40:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [28]:
!nvidia-smi -q


==============NVSMI LOG==============

Timestamp                                 : Fri Feb 20 00:40:26 2026
Driver Version                            : 580.82.07
CUDA Version                              : 13.0

Attached GPUs                             : 1
GPU 00000000:00:04.0
    Product Name                          : Tesla T4
    Product Brand                         : NVIDIA
    Product Architecture                  : Turing
    Display Mode                          : Requested functionality has been deprecated
    Display Attached                      : Yes
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/

In [29]:
!nvidia-smi --query-gpu=name,memory.total,memory.used,memory.free --format=csv

name, memory.total [MiB], memory.used [MiB], memory.free [MiB]
Tesla T4, 15360 MiB, 0 MiB, 14913 MiB


In [30]:
!nvidia-smi -l 1

Fri Feb 20 00:40:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             27W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [31]:
!nvidia-smi pmon -c 1

# gpu         pid   type     sm    mem    enc    dec    jpg    ofa    command 
# Idx           #    C/G      %      %      %      %      %      %    name 
    0          -     -      -      -      -      -      -      -    -              


In [32]:
!nvidia-smi dmon

# gpu    pwr  gtemp  mtemp     sm    mem    enc    dec    jpg    ofa   mclk   pclk 
# Idx      W      C      C      %      %      %      %      %      %    MHz    MHz 
    0     27     44      -      0      0      0      0      0      0   5000    705 
    0     27     44      -      0      0      0      0      0      0   5000    705 
    0     14     43      -      0      0      0      0      0      0    405    300 


Q3: Debug common CUDA errors

A) Zero Output Fix

In [33]:
%%writefile zero_output.cu
#include<stdio.h>

__global__ void f1(){
printf("Hello from GPU\n");
}

int main(){
f1<<<1,1>>>();
cudaDeviceSynchronize();
return 0;
}

Writing zero_output.cu


In [34]:
!nvcc -arch=sm_75 zero_output.cu -o zero_output

In [35]:
!./zero_output

Hello from GPU


B) Incorrect Indexing Fix

In [36]:
%%writefile incorrect_index.cu
#include<stdio.h>

__global__ void f2(){
int t=blockIdx.x*blockDim.x+threadIdx.x;
printf("Thread ID: %d\n",t);
}

int main(){
f2<<<2,4>>>();
cudaDeviceSynchronize();
return 0;
}

Writing incorrect_index.cu


In [37]:
!nvcc -arch=sm_75 incorrect_index.cu -o incorrect_index


In [38]:
!./incorrect_index

Thread ID: 4
Thread ID: 5
Thread ID: 6
Thread ID: 7
Thread ID: 0
Thread ID: 1
Thread ID: 2
Thread ID: 3


C) PTX Architecture Fix (compile command)

In [39]:
!nvcc -arch=sm_75 incorrect_index.cu -o incorrect_index

Q4: GPU kernel execution and thread indexing

In [40]:
%%writefile thread_index.cu
#include<stdio.h>

__global__ void g1(){
int global_id=blockIdx.x*blockDim.x+threadIdx.x;
printf("Hello from GPU thread %d\n",global_id);
}

int main(){
printf("Launching kernel...\n");
g1<<<1,8>>>();
cudaDeviceSynchronize();
printf("Back to CPU\n");
return 0;
}

Writing thread_index.cu


In [41]:
!nvcc -arch=sm_75 thread_index.cu -o thread_index

In [42]:
!./thread_index

Launching kernel...
Hello from GPU thread 0
Hello from GPU thread 1
Hello from GPU thread 2
Hello from GPU thread 3
Hello from GPU thread 4
Hello from GPU thread 5
Hello from GPU thread 6
Hello from GPU thread 7
Back to CPU


Q5: Host and Device memory separation

In [43]:
%%writefile memory_demo.cu
#include<stdio.h>
#include<cuda_runtime.h>

#define N 5

__global__ void p1(int *d){
int t=threadIdx.x;
printf("GPU Thread %d: Value = %d\n",t,d[t]);
}

int main(){
int h[N]={10,20,30,40,50};
int *g;
int s=N*sizeof(int);

cudaMalloc((void**)&g,s);
cudaMemcpy(g,h,s,cudaMemcpyHostToDevice);

p1<<<1,N>>>(g);
cudaDeviceSynchronize();

cudaMemcpy(h,g,s,cudaMemcpyDeviceToHost);

printf("\nBack on CPU:\n");
for(int i=0;i<N;i++)
printf("%d ",h[i]);

printf("\n");

cudaFree(g);
return 0;
}

Writing memory_demo.cu


In [44]:
!nvcc -arch=sm_75 memory_demo.cu -o memory_demo


In [45]:
!./memory_demo


GPU Thread 0: Value = 10
GPU Thread 1: Value = 20
GPU Thread 2: Value = 30
GPU Thread 3: Value = 40
GPU Thread 4: Value = 50

Back on CPU:
10 20 30 40 50 


Q6: Compare CPU times of List/tuple with Numpy arrays.

In [46]:
import time
import numpy as np

n=10_000_000
x=list(range(n))
y=list(range(n))

s=time.time()
z=[x[i]+y[i] for i in range(n)]
e=time.time()
print("Python List Time:",e-s)

x1=np.arange(n)
y1=np.arange(n)

s=time.time()
z1=x1+y1
e=time.time()
print("NumPy Time:",e-s)

Python List Time: 0.9606425762176514
NumPy Time: 0.03117823600769043
